In [ ]:
pwd

'/content'

In [ ]:
!pip install keras==2.2.4
!pip install tensorflow==2.5.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/base_command.py", line 180, in _main
    status = self.run(options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/req_command.py", line 199, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py", line 319, in run
    reqs, check_supported_wheels=not options.target_dir
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 128, in resolve
    requirements, max_rounds=try_to_avoid_resolution_too_deep
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/resolvelib/resolvers.py", line 473, in resolve
    state = resolution.resolve(requirements, max_rounds=max_rounds)
  File "/usr/local/lib/python3.7/dist-pack

IMAGE PREPROCESSING


import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='qeDqwC79iRzC9aCmvZa-e7O_s53nbgt-zbF2fG-9aIGP',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'imageclassification-donotdelete-pr-g7epqhkbzlxssg'
object_key = 'Classification of Arrhythmia by Using Deep Learning with 2-D ECG Spectral Image Representation.zip'

streaming_body_5 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
#unzipping your data file
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_5.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [ ]:
ls

sample_data/


In [ ]:
pwd

'/content'

import os
filenames=os.listdir('/home/wsuser/work/data/data/train')

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# setting parameters for data augmentation for training data
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)

In [ ]:
# data augmentation to the testing data
test_datagen=ImageDataGenerator(rescale=1./255)

#performing data augmentation to train data
x_train=train_datagen.flow_from_directory(directory=r'/home/wsuser/work/data/data/train',target_size=(64,64),batch_size=32,class_mode='categorical')

MODEL BUILDING

In [ ]:
import numpy as np
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

from tensorflow.keras.layers import Dense,Flatten

from tensorflow.keras.layers import Conv2D,MaxPooling2D
import tensorflow.keras

In [ ]:
model=Sequential()


Adding CNN Layers


In [ ]:
model.add(Conv2D(32,(3,3),input_shape=(64,64,3),activation='relu')) #Activation Function
model.add(MaxPooling2D(pool_size=(2,2))) #Downsampling Purposes
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten()) #Flatten the dimensions of the image


ADDING DENSE LAYERS

In [ ]:
model.add(Dense(32))# Deeply connected neural network layers
model.add(Dense(6,activation='softmax'))#Output layer with 6 neurons

In [ ]:
#Summary of the model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6272)              0         
_________________________________________________________________
dense (Dense)                (None, 32)                200736    
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 1

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])


TRAINING THE MODEL

model.fit_generator(generator=x_train,steps_per_epoch = len(x_train), epochs=10, validation_data=x_test,validation_steps = len(x_test))


SAVING THE MODEL

In [ ]:
ls

sample_data/


In [ ]:
pwd

'/content'

In [ ]:
model.save('ECG_Classification.h5')

In [ ]:
ls

ECG_Classification.h5  sample_data/


In [ ]:
!tar -zcvf ECG-Arrythmia-model_new.tgz ECG_Classification.h5

ECG_Classification.h5


In [ ]:
ls -1

ECG-Arrythmia-model_new.tgz
ECG_Classification.h5
sample_data/


Test the model

In [ ]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

#Loading of image
img=image.load_img(r'/home/wsuser/work/data/test/Premature Ventricular Contractions/fig_5660.png',target_size=(64,64))


![image.png](vvv)

In [ ]:
#Converting image to array

x

#Changing the shape
x=np.expand_dims(x,axis=0)

x_train.class_indices

IBM Deployment


In [ ]:
#Installing amchine learning service
!pip install watson-machine-learning-client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#Replace the credentials that you got from Watson Machine Learning Service
from ibm_watson_machine_learning import APIClient
wml_credentials={
    "url":"https://us-south.ml.cloud.ibm.com",
    "apikey":"1BsYiy1byg98ByElNCxXh01pwMQPKW5pEqxaD464Bcfz"
}
client=APIClient(wml_credentials)

client

client=APIClient(wml_credentials)

In [ ]:

def guid_from_space_name(client,space_name):
    space=client.spaces.get_details()
    #print(space)
    return(next(item for item in space['resources'] if item['entity']["name"]==space_name)['metadata']['id'])

In [ ]:
space_uid=guid_from_space_name(client, 'Newsace')
print("space UID="+ space_uid)

NameError: ignored

In [ ]:
lient.set.default_space(space_uid)

NameError: ignored

In [ ]:
client.software_specifications.list(200)

software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
software_spec_uid

In [ ]:
model_details = client.repository.store_model(model='ECG-Arrythmia-model_new.tgz',meta_props={
    client.repository.ModelMetaNames.NAME:"CNN",
    client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid
})

NameError: ignored

In [ ]:
model_id=client.repository.get_model_uid(model_details)

NameError: ignored

In [ ]:
model_id

NameError: ignored